<a href="https://colab.research.google.com/github/yvainetian/CV_learning/blob/master/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task4 模型训练与验证

### 4.1 学习目标

    * 理解验证集的作用，并使用训练集和验证集完成训练    
    * 学会使用Pytorch环境下的模型读取和加载，并了解调参流程






## 4.2 构造验证集
    深度学习模型在不断的训练过程中易造成过拟合，模型对训练集上的数据有很好的解释能力，但在测试集上表现较差，即学习的模型泛化能力差。
    随着模型复杂度和模型训练轮数的增加，CNN模型在训练集上的误差会降低，但在测试集上的误差会逐渐降低，然后逐渐升高，因此一味追求训练集精度并非最终目的。

    导致模型过拟合的情况有很多种原因，其中最为常见的情况是模型复杂度（Model Complexity ）太高，导致模型学习到了训练数据的方方面面，学习到了一些细枝末节的规律。（用正则化矫正模型复杂度）
    解决上述问题最好的解决方法：构建一个与测试集尽可能分布一致的样本集（可称为验证集），在训练过程中不断验证模型在验证集上的精度，并以此控制模型的训练。

### 4.2.1 模型评估方式
    * 训练集（Train Set）：模型用于训练和调整模型参数
    * 验证集（Validation Set）：用来验证模型精度和调整模型超参数
    * 测试集（Test Set）：验证模型的泛化能力

### 4.2.2 验证集的划分

* 留出法（Hold-Out）  
        直接将训练集划分成两部分，新的训练集和验证集。这种划分方式的优点是最为直接简单；缺点是只得到了一份验证集，有可能导致模型在验证集上过拟合。留出法应用场景是数据量比较大的情况。
        它对训练集、验证集和测试集的样本数比例，还有分割后数据的分布是否和原始数据集的分布相同等因素比较敏感，不同的划分会得到不同的最优模型。

* 交叉验证法（Cross Validation，K—fold or Leave-one-out）    
       （ k 折交叉验证通过对 k 个不同分组训练的结果进行平均来减少方差，因此模型的性能对数据的划分就不那么敏感，验证集精度比较可靠，训练K次可以得到K个有多样性差异的模型；缺点是需要训练K次，不适合数据量很大的情况。）
        1. 不重复抽样将原始数据随机分为 k 份。
        2. 每一次挑选其中 1 份作为测试集，剩余 k-1 份作为训练集用于模型训练。
        3. 重复第二步 k 次，这样每个子集都有一次机会作为测试集，其余机会作为训练集。在每个训练集上训练后得到一个模型，用这个模型在相应的测试集上测试，计算并保存模型的评估指标，
        4. 计算 k 组测试结果的平均值作为模型精度的估计，并作为当前 k 折交叉验证下模型的性能指标。
    
* 自助采样法（BootStrap）
        在含有 m 个样本的数据集中，进行 m 次有放回地随机抽样，组成的新数据集作为训练集。
        这种方法，有的样本会被多次采样，也会有一次都没有被选择过的样本，原数据集中大概有 36.8% 的样本不会出现在新组数据集中，这些没有被选择过的数据作为验证集。

        优点是训练集的样本总数和原数据集一样都是 m，并且仍有约 1/3 的数据不被训练而可以作为测试集，对于样本数少的数据集，就不用再由于拆分得更小而影响模型的效果。
        缺点是这样产生的训练集的数据分布和原数据集的不一样了，会引入估计偏差。




## 4.3 模型训练与验证 

    * 构造训练集和验证集；
    * 每轮进行训练和验证，并根据最优验证集精度保存模型。



In [0]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=10, 
    shuffle=True, 
    num_workers=10, 
)
    
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=10, 
    shuffle=False, 
    num_workers=10, 
)

model = SVHN_Model1()
criterion = nn.CrossEntropyLoss (size_average=False)   # CrossEntropy是LogSoftmax和NLLLoss的结合，适用于多分类问题 https://mfy.world/deep-learning/pytorch/pytorchnotes-lossfunc/
optimizer = torch.optim.Adam(model.parameters(), 0.001)  # 优化算法/ Adam相当于 RMSprop + Momentum https://www.cnblogs.com/guoyaohua/p/8542554.html
best_loss = 1000.0

for epoch in range(20):
    print('Epoch: ', epoch)

    train(train_loader, model, criterion, optimizer, epoch)
    val_loss = validate(val_loader, model, criterion)
    
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), './model.pt')

In [0]:
def train(train_loader, model, criterion, optimizer, epoch):
    # 切换模型为训练模式
    model.train()

    for i, (input, target) in enumerate(train_loader):
        c0, c1, c2, c3, c4, c5 = model(data[0])
        loss = criterion(c0, data[1][:, 0]) + \
                criterion(c1, data[1][:, 1]) + \
                criterion(c2, data[1][:, 2]) + \
                criterion(c3, data[1][:, 3]) + \
                criterion(c4, data[1][:, 4]) + \
                criterion(c5, data[1][:, 5])
        loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [0]:
def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            c0, c1, c2, c3, c4, c5 = model(data[0])
            loss = criterion(c0, data[1][:, 0]) + \
                    criterion(c1, data[1][:, 1]) + \
                    criterion(c2, data[1][:, 2]) + \
                    criterion(c3, data[1][:, 3]) + \
                    criterion(c4, data[1][:, 4]) + \
                    criterion(c5, data[1][:, 5])
            loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

### *  pytorch中model.eval()和“with torch.no_grad()区别 
    在PyTorch中进行validation时，会使用model.eval()切换到测试模式：
    主要用于通知dropout层和batchnorm层在train和val模式间切换
    在train模式下，dropout网络层会按照设定的参数p设置保留激活单元的概率（保留概率=p); batchnorm层会继续计算数据的mean和var等参数并更新。
    在val模式下，dropout层会让所有的激活单元都通过，而batchnorm层会停止计算和更新mean和var，直接使用在训练阶段已经学出的mean和var值。
    该模式不会影响各层的gradient计算行为，即gradient计算和存储与training模式一样，只是不进行反传（backprobagation）

    而with torch.zero_grad()则主要是用于停止autograd模块的工作，以起到加速和节省显存的作用，具体行为就是停止gradient计算，从而节省了GPU算力和显存，但是并不会影响dropout和batchnorm层的行为。


## 4.4 模型保存与加载

```
#torch.save(model_object.state_dict(), 'model.pt')
model.load_state_dict(torch.load(' model.pt')) 
```



## 4.5 模型调参
    贪心算法 https://www.jianshu.com/p/ab89df9759c8
    网格调参 https://blog.csdn.net/weixin_43172660/article/details/83032029
    贝叶斯调参 https://blog.csdn.net/linxid/article/details/81189154


* 模型泛化能力：从简单CNN模型出发，增加模型复杂度减小损失，但同时增加数据扩增方法，直至验证集精度不变。
